In [1]:
import numpy as np
import nilearn as ni
from matplotlib import pyplot as plt
from nilearn import plotting,image
from sklearn.preprocessing import MinMaxScaler,LabelEncoder
import pandas as pd
from tqdm import tqdm
import h5py
import os
from glob import glob
os.environ["CUDA_VISIBLE_DEVICES"]="2"
from keras.utils import to_categorical

%matplotlib inline

C:\Users\Naveen\Anaconda2\envs\Python_35\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


# Load Healthy Data

In [5]:
path_healthy='E:\\Projects\\fmri_analysis\\Schizophrencx\\Healthy'

In [6]:
path_schizophrenic='E:\\Projects\\fmri_analysis\\Schizophrencx\\Schizophrenic'

# Get all .hdr filenames from subdirectories Healthy

In [7]:
filenames=[]
# lp=[]
for x in os.walk(path_healthy):
    if 'final' in x[0]:
#         lp.append(x[0])
        filenames.append(glob(x[0]+'\\*.hdr')) 
        

In [8]:
# for i in range(len(filenames)):
#     print(len(filenames[i]))

In [9]:
data = [image.load_img(filename).get_data() for sub_filenames in filenames for filename in sub_filenames]

In [10]:
len(data),data[0].shape

(18900, (53, 63, 46))

In [11]:
data=np.array(data)

# Timeseries Schizophrenic healthy

In [12]:
l=[]
for sub_filenames in tqdm( filenames ):
    l.append(image.load_img(sub_filenames).get_data())
    
    
#     for filename in sub_filenames:
#         data = [image.load_img(filename).get_data()] 

100%|██████████| 135/135 [00:54<00:00,  2.49it/s]


In [13]:
len(filenames)

135

In [14]:
l=np.array(l)

In [15]:
l.shape

(135, 53, 63, 46, 140)

In [16]:
l=np.rollaxis(l,axis=-1,start=1)

In [17]:
l.shape

(135, 140, 53, 63, 46)

# Get all .hdr filenames from subdirectories Schizophrenic

In [18]:
filenames_schizophrenic=[]
for x in os.walk(path_schizophrenic):
    if 'final' in x[0]:
        filenames_schizophrenic.append(glob(x[0]+'\\*.hdr'))      

In [19]:
data_schizophrenic = [image.load_img(filename).get_data() for sub_filenames in filenames_schizophrenic for filename in sub_filenames]

In [20]:
len(data_schizophrenic),data_schizophrenic[0].shape

(19040, (53, 63, 46))

In [21]:
data_schizophrenic=np.array(data_schizophrenic)

In [22]:
data_schizophrenic.shape

(19040, 53, 63, 46)

# TimeSeries Schizophrenic Schizophrenic data

In [23]:
l_schizophrenic=[]
for sub_filenames in tqdm(filenames_schizophrenic):
    l_schizophrenic.append(image.load_img(sub_filenames).get_data())

100%|██████████| 136/136 [00:55<00:00,  2.45it/s]


In [24]:
l_schizophrenic=np.array(l_schizophrenic)

In [25]:
l_schizophrenic.shape

(136, 53, 63, 46, 140)

In [26]:
l_schizophrenic=np.rollaxis(l_schizophrenic,axis=-1,start=1)

In [27]:
l_schizophrenic.shape

(136, 140, 53, 63, 46)

# Saving Data to file

In [28]:
h5_obj=h5py.File('schiz_healthy.h5','w')

In [29]:
h5_obj.create_dataset('healthy',data=data)

<HDF5 dataset "healthy": shape (18900, 53, 63, 46), type "<i2">

In [30]:
h5_obj.close()

In [31]:
h5_obj_schizo=h5py.File('schiz_schizo.h5','w')

In [32]:
h5_obj_schizo.create_dataset('schizo',data=data_schizophrenic)

<HDF5 dataset "schizo": shape (19040, 53, 63, 46), type "<i2">

In [33]:
h5_obj_schizo.close()

# Saving Timeseries Data

In [34]:
h5_obj=h5py.File('schiz_time_healthy.h5','w')

In [35]:
h5_obj.create_dataset('healthy',data=l)

<HDF5 dataset "healthy": shape (135, 140, 53, 63, 46), type "<i2">

In [36]:
h5_obj.close()

In [37]:
h5_obj_schizo=h5py.File('schiz_time_schizo.h5','w')

In [38]:
h5_obj_schizo.create_dataset('schizo',data=l_schizophrenic)

<HDF5 dataset "schizo": shape (136, 140, 53, 63, 46), type "<i2">

In [39]:
h5_obj_schizo.close()

# load data 

In [40]:
h5_rd=h5py.File('schiz_healthy.h5','r')

In [41]:
h5_rd.keys()

KeysView(<HDF5 file "schiz_healthy.h5" (mode r)>)

In [42]:
data_h=np.array(h5_rd['healthy'])

In [43]:
h5_rd.close()

In [44]:
data_h.shape

(18900, 53, 63, 46)

In [45]:
h5_rd=h5py.File('schiz_schizo.h5','r')

In [46]:
h5_rd.keys()

KeysView(<HDF5 file "schiz_schizo.h5" (mode r)>)

In [47]:
data_s=np.array(h5_rd['schizo'])

In [48]:
h5_rd.close()

In [49]:
data_s.shape

(19040, 53, 63, 46)

In [50]:
X=np.append(data_h,data_s,axis=0)

In [51]:
X.shape

(37940, 53, 63, 46)

In [52]:
y=np.zeros((X.shape[0]))

In [53]:
y[data_h.shape[0]:]=1

In [54]:
#one-hot encoding
y_encode=to_categorical(y)

In [55]:
y_encode

array([[1., 0.],
       [1., 0.],
       [1., 0.],
       ...,
       [0., 1.],
       [0., 1.],
       [0., 1.]], dtype=float32)

In [56]:
path="E:\\Projects\\fmri_analysis\\Schizophrencx\\000900369824_49_h_f\\AudOdd4\\final\\"

In [57]:
plotting.plot_img(image.load_img(path+"swarf0004.img"))

<IPython.core.display.Javascript object>

# Importing required libraries (Keras framework)

In [58]:
from keras.models import Sequential,Model
from keras.layers import Input,Conv3D, UpSampling3D, MaxPooling3D
from keras.optimizers import Adam, SGD, Adadelta, RMSprop, Nadam
from keras.utils import multi_gpu_model

# Constructing Convolutional Autoencoder

In [59]:
#model = Sequential()
input_img=Input(shape=(52,63,45,1))
#input_img = Input(shape=(53,63,46,1))

#Encoder
x=Conv3D(8,(3,3,3),padding='same',activation = 'relu')(input_img)
x=MaxPooling3D((2,3,3))(x)
x=Conv3D(8,(3,3,3),padding='same',activation = 'relu')(x)
encoded=MaxPooling3D((2,3,3),padding = 'same')(x)

#Decoder
x=Conv3D(8,(3,3,3),padding='same',activation = 'relu')(encoded)
x=UpSampling3D((2,3,3))(x)
x=Conv3D(8,(3,3,3),padding='same',activation = 'relu')(x)
x=UpSampling3D((2,3,3))(x)
decoded=Conv3D(1,(2,2,2),padding='same')(x)

autoencoder=Model(input_img,decoded)

In [60]:
autoencoder.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 52, 63, 45, 1)     0         
_________________________________________________________________
conv3d_1 (Conv3D)            (None, 52, 63, 45, 8)     224       
_________________________________________________________________
max_pooling3d_1 (MaxPooling3 (None, 26, 21, 15, 8)     0         
_________________________________________________________________
conv3d_2 (Conv3D)            (None, 26, 21, 15, 8)     1736      
_________________________________________________________________
max_pooling3d_2 (MaxPooling3 (None, 13, 7, 5, 8)       0         
_________________________________________________________________
conv3d_3 (Conv3D)            (None, 13, 7, 5, 8)       1736      
_________________________________________________________________
up_sampling3d_1 (UpSampling3 (None, 26, 21, 15, 8)     0         
__________

In [61]:
encoder = Model(input_img, encoded)

In [62]:
encoder.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 52, 63, 45, 1)     0         
_________________________________________________________________
conv3d_1 (Conv3D)            (None, 52, 63, 45, 8)     224       
_________________________________________________________________
max_pooling3d_1 (MaxPooling3 (None, 26, 21, 15, 8)     0         
_________________________________________________________________
conv3d_2 (Conv3D)            (None, 26, 21, 15, 8)     1736      
_________________________________________________________________
max_pooling3d_2 (MaxPooling3 (None, 13, 7, 5, 8)       0         
Total params: 1,960
Trainable params: 1,960
Non-trainable params: 0
_________________________________________________________________


In [63]:
autoencoder.compile(optimizer='adam',loss='mse')

# Resizing image (omitting pixels)

In [64]:
X = X[:,0:52,:,0:45].reshape(X.shape[0],52,63,45,1)
X.shape

(37940, 52, 63, 45, 1)

# Image attributes

In [65]:
img_h=X.shape[1]
img_w=X.shape[2]
img_d=X.shape[3]
nb_classes=2

# Training autoencoder

In [68]:
hist_ae = autoencoder.fit(X,X,epochs=240,batch_size=256,shuffle=True,validation_split=0.33,verbose=2)

Train on 25419 samples, validate on 12521 samples
Epoch 1/240
 - 296s - loss: 3135.6207 - val_loss: 2193.1503
Epoch 2/240
 - 296s - loss: 3207.1861 - val_loss: 2277.0840
Epoch 3/240
 - 297s - loss: 3086.8334 - val_loss: 2307.0968
Epoch 4/240
 - 296s - loss: 3119.3457 - val_loss: 2200.5232
Epoch 5/240
 - 297s - loss: 3095.7760 - val_loss: 2186.2089
Epoch 6/240
 - 296s - loss: 3103.2362 - val_loss: 2126.9964
Epoch 7/240
 - 296s - loss: 3059.0155 - val_loss: 2184.0303
Epoch 8/240
 - 295s - loss: 3057.2962 - val_loss: 2135.8815
Epoch 9/240
 - 294s - loss: 3055.3805 - val_loss: 2173.9140
Epoch 10/240
 - 296s - loss: 3034.4717 - val_loss: 2111.2300
Epoch 11/240
 - 296s - loss: 3064.5473 - val_loss: 2079.5884
Epoch 12/240
 - 295s - loss: 3017.1654 - val_loss: 2095.1265
Epoch 13/240
 - 295s - loss: 3033.1110 - val_loss: 2081.2338
Epoch 14/240
 - 295s - loss: 2971.3592 - val_loss: 2062.3142
Epoch 15/240
 - 294s - loss: 3053.0385 - val_loss: 2054.3265
Epoch 16/240
 - 295s - loss: 2982.4665 - val

In [72]:

# summarize history for loss for first 50 epochs
plt.plot(hist_ae.history['loss'])
plt.plot(hist_ae.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

# Saving data to file

In [74]:
#finding latent representation
fmri_encoded_x_data = encoder.predict(X)
fmri_encoded_x_data.shape

(37940, 13, 7, 5, 8)

In [75]:
h5_object = h5py.File('fmri_encoded_data_130718_3D.h5', 'w')
h5_object_y = h5py.File('fmri_label_data_130718_3D.h5', 'w')

In [76]:
h5_object.create_dataset('fmri_encoded_data',data=fmri_encoded_x_data)
h5_object_y.create_dataset('fmri_label',data=y_encode)
h5_object.close()
h5_object_y.close()

# Loading data from file

In [120]:
h5f = h5py.File('fmri_encoded_data_130718_3D.h5','r')
h5fy= h5py.File('fmri_label_data_130718_3D.h5','r')
X = np.array(h5f['fmri_encoded_data'])
y = np.array(h5fy['fmri_label'])

# Pre-processing and PCA

In [121]:
X_data = X.reshape(37940,13*7*5*8)

In [130]:
X_data.shape

array([[1., 0.],
       [1., 0.],
       [1., 0.],
       ...,
       [0., 1.],
       [0., 1.],
       [0., 1.]], dtype=float32)

In [123]:
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
pca = PCA(n_components = 10)
pca.fit(X_data)

PCA(copy=True, iterated_power='auto', n_components=10, random_state=None,
  svd_solver='auto', tol=0.0, whiten=False)

In [124]:
var = np.array(pca.explained_variance_ratio_)
np.sum(var)
#retaining 95% of variance

0.95514864

In [131]:
X_data = pca.fit_transform(X_data)
X_data.shape
X_transform = X_data
#y = np.argmax(y,axis = 1)

array([0, 0, 0, ..., 1, 1, 1], dtype=int64)

In [135]:
X_train, X_test, y_train, y_test = train_test_split(X_transform,y, test_size=0.33,shuffle=True)

# Logistic Regression

In [136]:
from sklearn.linear_model import LogisticRegression

In [137]:
lr = LogisticRegression()
lr.fit(X_train,y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [138]:
lr.score(X_test,y_test)

0.643079626227937

# GDA (Linear Discriminant Analysis)

In [143]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA,QuadraticDiscriminantAnalysis as QDA

In [145]:
lda = LDA()
lda.fit(X_train,y_train)

LinearDiscriminantAnalysis(n_components=None, priors=None, shrinkage=None,
              solver='svd', store_covariance=False, tol=0.0001)

In [146]:
lda.score(X_test,y_test)

0.6446769427362031

# GDA with class-specific covariance matrices (Quadratic Discriminant Analysis)

In [148]:
qda = QDA()
qda.fit(X_train,y_train)

QuadraticDiscriminantAnalysis(priors=None, reg_param=0.0,
               store_covariance=False, store_covariances=None, tol=0.0001)

In [149]:
qda.score(X_test,y_test)

0.7164763197827649

# Decision tree

In [150]:
from sklearn.tree import DecisionTreeClassifier

In [153]:
dt = DecisionTreeClassifier()
dt.fit(X_train,y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [154]:
dt.score(X_test,y_test)

0.991374490855363